# Example notebook

In [1]:
%load_ext autoreload
%autoreload 2
from multi_condition_comparisions.tl.de import StatsmodelsDE
import scanpy as sc

/home/sturm/apps/micromamba/envs/multi-condition-comparisons/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata = sc.datasets.pbmc68k_reduced()
adata.obs

,bulk_labels,n_genes,percent_mito,n_counts,S_score,G2M_score,phase,louvain
index,,,,,,,,
AAAGCCTGGCTAAC-1,CD14+ Monocyte,1003,0.023856,2557.0,-0.119160,-0.816889,G1,1
AAATTCGATGCACA-1,Dendritic,1080,0.027458,2695.0,0.067026,-0.889498,S,1
AACACGTGGTCTTT-1,CD56+ NK,1228,0.016819,3389.0,-0.147977,-0.941749,G1,3
AAGTGCACGTGCTA-1,CD4+/CD25 T Reg,1007,0.011797,2204.0,0.065216,1.469291,G2M,9
ACACGAACGGAGTG-1,Dendritic,1178,0.017277,3878.0,-0.122974,-0.868185,G1,2
...,...,...,...,...,...,...,...,...
TGGCACCTCCAACA-8,Dendritic,1166,0.008840,3733.0,-0.124456,-0.867484,G1,2
TGTGAGTGCTTTAC-8,Dendritic,1014,0.022068,2311.0,-0.298056,-0.649070,G1,1
TGTTACTGGCGATT-8,CD4+/CD25 T Reg,1079,0.012821,3354.0,0.216895,-0.527338,S,0


In [3]:
mod = StatsmodelsDE(adata, "~ n_genes + bulk_labels*phase")

In [4]:
mod.fit()

  7%|▋         | 51/765 [00:00<00:03, 187.62it/s]

100%|██████████| 765/765 [00:08<00:00, 86.69it/s] 


### Test a simple contrast

In [5]:
mod.test_contrasts({"G2M_vs_S": mod.contrast("phase", "S", "G2M")})

100%|██████████| 765/765 [00:02<00:00, 365.20it/s]


,pvalue,tvalue,sd,fold_change,contrast
variable,,,,,
HMGB2,1.2048152733819692e-110,-27.221294,0.210698,-5.735484,G2M_vs_S
STMN1,4.383355621869748e-32,-12.430114,0.290148,-3.606578,G2M_vs_S
CALM3,7.642249886520726e-13,-7.308989,0.350155,-2.559276,G2M_vs_S
CCR10,2.519033257850441e-07,-5.209515,0.387708,-2.019772,G2M_vs_S
NINJ2,3.2189242861750864e-06,-4.695796,0.407821,-1.915043,G2M_vs_S
...,...,...,...,...,...
RAB10,0.997309546551229,0.003373,0.383556,0.001294,G2M_vs_S
GZMB,0.9975430768266769,-0.003080,0.298777,-0.000920,G2M_vs_S
CNPY3,0.998027988877058,-0.002472,0.357454,-0.000884,G2M_vs_S


### Build a more complex contrast

In [6]:
mod.test_contrasts(
    {
        "G2M vs S in Dendritic cells": mod.cond(bulk_labels="Dendritic", phase="G2M")
        - mod.cond(bulk_labels="Dendritic", phase="S")
    }
)

  0%|          | 0/765 [00:00<?, ?it/s]

100%|██████████| 765/765 [00:00<00:00, 1386.26it/s]


,pvalue,tvalue,sd,fold_change,contrast
variable,,,,,
HMGB2,7.379049356876103e-116,28.146733,0.205242,5.776896,G2M vs S in Dendritic cells
STMN1,1.1452526372660046e-35,13.225603,0.282635,3.738015,G2M vs S in Dendritic cells
CALM3,1.9772873146480225e-14,7.824348,0.341087,2.668783,G2M vs S in Dendritic cells
GTF3C6,8.317869063811358e-09,5.835783,0.417569,2.436839,G2M vs S in Dendritic cells
IL32,2.3059965204793834e-08,5.654785,0.296670,1.677604,G2M vs S in Dendritic cells
...,...,...,...,...,...
DTD1,0.9940167079027864,-0.007502,0.398430,-0.002989,G2M vs S in Dendritic cells
IL16,0.9964044894037746,0.004508,0.385664,0.001739,G2M vs S in Dendritic cells
HLA-DQB1,0.9967242650678589,0.004107,0.292488,0.001201,G2M vs S in Dendritic cells


### Test an arbitrary contrast vector 

In [7]:
mod.test_contrasts(
    {
        "G2M vs S in Dendritic cells": [
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            1.0,
            -1.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            1.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            -1.0,
        ]
    }
)

  0%|          | 0/765 [00:00<?, ?it/s]

100%|██████████| 765/765 [00:00<00:00, 3535.54it/s]


,pvalue,tvalue,sd,fold_change,contrast
variable,,,,,
HMGB2,7.379049356876103e-116,28.146733,0.205242,5.776896,G2M vs S in Dendritic cells
STMN1,1.1452526372660046e-35,13.225603,0.282635,3.738015,G2M vs S in Dendritic cells
CALM3,1.9772873146480225e-14,7.824348,0.341087,2.668783,G2M vs S in Dendritic cells
GTF3C6,8.317869063811358e-09,5.835783,0.417569,2.436839,G2M vs S in Dendritic cells
IL32,2.3059965204793834e-08,5.654785,0.296670,1.677604,G2M vs S in Dendritic cells
...,...,...,...,...,...
DTD1,0.9940167079027864,-0.007502,0.398430,-0.002989,G2M vs S in Dendritic cells
IL16,0.9964044894037746,0.004508,0.385664,0.001739,G2M vs S in Dendritic cells
HLA-DQB1,0.9967242650678589,0.004107,0.292488,0.001201,G2M vs S in Dendritic cells
